[Table of Contents](table_of_contents.ipynb)

# Appendix A. Common Attitude Representations

[Introduction](#Introduction)

## Introduction

A point in 3D space can be thought of as a vector pointing from the origin to the point. This vector, when described in a frame of reference, is fully parameterized by three scalars. Being a vector, it inherits all of the beneficial mathematical properties of a vector space, including associativity and commutativity of addition, the identity and inverse elements, and scalar multiplication.

Attitude, on the other hand, is not so elegant. While it is true, that there are only three degrees of freedom to describe orientation, none of the representation methods form a global, minimum representation. Some behave like vector spaces, but are only local charts (e.g. Euler anges, Rodriguez) and others are global but are embedded in a higher dimensional space (e.g. rotation matrices, unit quaternions).

Here we will explore some of the most common methods for representing attitude and touch on their pros and cons. Much of this material is based on Shuster's work in
 * Shuster, M. D. (1993). A Survey of attitude representations. The Journal of the Astronautical Sciences, 1(4), 439–517.

## The $SO(3)$ Manifold

In [ ]:
%matplotlib notebook
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle
from mpl_toolkits import mplot3d
from matplotlib.widgets import Slider

class Attitude:
    # This class will be used to convert between the many methods of representing attitude.
    # The base internal representation will be a rotation matrix, as it has the fewest
    # caveats and conversion to and from the rotation matrix is well understood.
    def __init__(self, R=np.eye(3)):
        self.R = R
        self.init_visualization()
    
    @staticmethod
    def identity():
        return Attitude()
    
    def set_rotation_matrix(self, R):
        self.R = R
        self.update_vis()
        
    def set_euler_angles(self, phi, theta, psi, order='321'):
        # Note: these are passive notations
        c = np.cos
        s = np.sin
        rot_1 = lambda x: np.array([[1,     0,    0],
                                    [0,  c(x), s(x)],
                                    [0, -s(x), c(x)]])
        rot_2 = lambda x: np.array([[c(x), 0, -s(x)],
                                    [   0, 1,     0],
                                    [s(x), 0,  c(x)]])
        rot_3 = lambda x: np.array([[ c(x), s(x), 0],
                                    [-s(x), c(x), 0],
                                    [   0,     0, 1]])
        rots = {"1":rot_1, "2":rot_2, "3":rot_3}
        order = list(order)
        R = np.eye(3)
        R = rots[order[0]](psi).dot(R)
        R = rots[order[1]](theta).dot(R)
        R = rots[order[2]](phi).dot(R)
        self.set_rotation_matrix(R)
        
    def init_visualization(self):
        # load and draw the plane
        self._plane = pickle.load(open("files/T38.p", "rb"))
        self._figure = plt.figure()
        self._axes = mplot3d.Axes3D(self._figure)
        scale = self._plane.points/np.max(self._plane.points).flatten()
        self._base_R = np.array([[0, 0, 1],
                                 [1, 0, 0],
                                 [0, -1, 0]])
        R = self.R.T.dot(self._base_R)
        vectors = np.einsum('ij,...j->...i', R, self._plane.vectors)/np.max(self._plane.points)
        self._faces = mplot3d.art3d.Poly3DCollection(vectors)
        self._faces.set_facecolor((0,0,1))
        self._faces.set_edgecolor((0.3, 0.3, 0.6))
        self._axes.add_collection3d(self._faces)
        self._axes.auto_scale_xyz(scale, scale, scale)
        self._axes.set_xlabel('North')
        self._axes.set_ylabel('East')
        self._axes.set_zlabel('Down')
        self._axes.invert_zaxis()
        
        # draw the inertial frame
        self._axes.quiver([0,0,0],[0,0,0],[0,0,0],#[-1,-1,-1],[1,1,1],[1,1,1],
                    [1,0,0],[0,1,0],[0,0,1], 
                    colors=['r','g','b','r','r','g','g','b','b'], pivot='tail')
        
        # make some interactive sliders
        ax_yaw = plt.axes([0.25, 0.1, 0.65, 0.03])
        self._yaw_slider = Slider(ax_yaw, 'Yaw', -np.pi, np.pi, valinit=0)
        self._yaw_slider.on_changed(self.slider_update)
        
        # save the axes for later reference
#         self._axes = axes

    def slider_update(self, val):
        yaw = self._yaw_slider.val
        self.set_euler_angles(0, 0, yaw)
        
    def update_vis(self):
        R = self.R.dot(self._base_R)
        vectors = np.einsum('ij,...j->...i', R, self._plane.vectors)/np.max(self._plane.points)
        self._faces.set_verts(vectors)
#         self._figure.canvas.draw_idle()
    
    def show(self):
        self._figure.show()
        

In [ ]:
att = Attitude()
att.show()

## Rotation Matrices

## Euler Angles

## Axis-Azimuth

## Unit Quaternion

The unit quaternion, sometimes called *Euler-Rodrigues symmetric parameters*, *Euler symmetric parameters*, or the *quaternion of rotation*, is a four-dimensional representation of attitude.

We shall represent quaternions using the Hamiltonian Notation as
$$
\mathbf{q} = q_w + q_x i + q_y j + q_z k = \begin{bmatrix}q_w & \bar{\mathbf{q}} \end{bmatrix}^\top
$$
where $q_w$ is commonly called the *scalar part* and $\bar{\mathbf{q}}$ the *vector part* of the quaternion. For the quaternion to represent an attitude, we must enforce the constraint that the quaternion be of unit norm, i.e

$$
\|\mathbf{q}\| = 1
$$

Here is a reasonable treatment of the difference between the [Hamiltonian and JPL conventions](https://fzheng.me/2017/11/12/quaternion_conventions_en/). Note that Shuster uses the JPL convention.

The set of all quaternions, $\mathbb{H}$, is analagous to the complex numbers, $\mathbb{C}$, where

$$
i^2 = j^2 = k^2 = ijk = -1.
$$

While unit quaternions do not suffer from a singularity, they are a double cover of $SO(3)$, where $\mathbf{q}$ and $-\mathbf{q}$ correspond to the same attitude.

Unit quaternions form a Lie group, with operator
$$
\mathbf{r} = \mathbf{p} \otimes \mathbf{q} = \begin{bmatrix}p_{w} & -\bar{\boldsymbol{p}}^{\top}\\
\bar{\boldsymbol{p}} & p_{w}{\bf I}+\left\lfloor \bar{\boldsymbol{p}}\right\rfloor 
\end{bmatrix}\begin{bmatrix}q_{w}\\
\bar{\boldsymbol{q}}
\end{bmatrix}
$$